In [1]:
# coding: utf-8
import gc
import pickle
import numpy as np
import pandas as pd
from itertools import combinations
from tqdm import tqdm

cols = [
    #'EngineVersion',
    #'AppVersion',
    #'AvSigVersion',
    'AVProductStatesIdentifier',
    'AVProductsInstalled',
    'SmartScreen',
    'Census_OEMModelIdentifier',
    'Census_ProcessorModelIdentifier',
    'Census_PrimaryDiskTotalCapacity',
    'Census_TotalPhysicalRAM',
    'Census_InternalPrimaryDiagonalDisplaySizeInInches',
    'Census_FirmwareVersionIdentifier'
]

In [2]:
gc.enable()

In [3]:
#train = pd.read_csv('../input/train.csv', usecols=cols, dtype=utils.DTYPES)
train = pd.read_pickle('../input/msmalware/train.pkl')
print("TRAIN LOADED")

TRAIN LOADED


In [4]:
cols = list(train.columns)

target = 'HasDetections'
cols.remove(target)

test = pd.read_pickle('../input/msmalware/test.pkl')
print("TEST LOADED")

TEST LOADED


In [5]:
new_cols = []

cols.remove('MachineIdentifier')

def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

In [6]:
cols

['ProductName',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'IsBeta',
 'RtpStateBitfield',
 'IsSxsPassiveMode',
 'DefaultBrowsersIdentifier',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'HasTpm',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsVer',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'AutoSampleOptIn',
 'PuaMode',
 'SMode',
 'IeVerIdentifier',
 'SmartScreen',
 'Firewall',
 'UacLuaenable',
 'Census_MDC2FormFactor',
 'Census_DeviceFamily',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_ProcessorClass',
 'Census_PrimaryDiskTotalCapacity',
 'Census_PrimaryDiskTypeName',
 'Census_SystemVolumeTotalCapacity',
 'Census_HasOpticalDiskDrive',
 'Census_TotalPhysicalR

In [7]:
len(cols)

81

In [8]:
def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [9]:
cols_t= [cols[i] for i in [21,41]]
cols_t

['OsSuite', 'Census_PrimaryDiskTotalCapacity']

In [10]:
#for col in tqdm(cols[42:]):
for col in tqdm(cols_t):
        print(col)
        new_feature = 'fe_{}_target_mean_enc'.format(col)
        trn, sub = target_encode(train[col], 
                             test[col], 
                             target=train[target], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
        train[new_feature] = trn
        test[new_feature] = sub

        new_cols.append(new_feature)

print("computed, now reducing size")

  0%|          | 0/2 [00:00<?, ?it/s]

OsSuite


 50%|█████     | 1/2 [00:06<00:06,  6.46s/it]

Census_PrimaryDiskTotalCapacity


100%|██████████| 2/2 [00:12<00:00,  6.48s/it]

computed, now reducing size


In [11]:
gc.collect()

185

In [12]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
train_new_2 = train[new_cols]
test_new_2 = test[new_cols]

train_new_2 = reduce_mem_usage(train_new_2)
test_new_2 =  reduce_mem_usage(test_new_2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Mem. usage decreased to 34.03 Mb (75.0% reduction)
Mem. usage decreased to 29.96 Mb (75.0% reduction)


In [14]:
train_new_2.head()

,fe_OsSuite_target_mean_enc,fe_Census_PrimaryDiskTotalCapacity_target_mean_enc
0,0.515137,0.491211
1,0.513184,0.499023
2,0.485352,0.541016
3,0.508301,0.479736
4,0.482422,0.494141


In [15]:
print("saving results")

train_new_2.to_pickle('train_target_mean_full_2.pkl')
test_new_2.to_pickle('test_target_mean_full_2.pkl')

saving results
